In [8]:
nb_parm = 'ADK/adk_short_bot'
date_run = '1231231231'
model = ''

In [9]:
import sys
import os, io

sys.path.append("/home/jovyan/notebooks")

## Do the task after this

In [10]:
ADK_SOURCE_DIR = nb_parm #/ADK/PKG_NAME 
ADK_TARGET_DIR = 'packaged_agent/' + nb_parm #/packaged_agentADK/PKG_NAME 
PKG_NAME = nb_parm.split('/')[1] #PKG_NAME 

print("ADK_SOURCE_DIR :", ADK_SOURCE_DIR)
print("ADK_TARGET_DIR :", ADK_TARGET_DIR)
print("PKG_NAME :", PKG_NAME)

ADK_SOURCE_DIR : ADK/adk_short_bot
ADK_TARGET_DIR : packaged_agent/ADK/adk_short_bot
PKG_NAME : adk_short_bot


### Register model

In [11]:
import subprocess
from pathlib import Path
import os
import shutil

# Step 1: Copy source folder to a clean new target location
SOURCE_DIR = Path("/tmp") / Path(ADK_SOURCE_DIR)
TARGET_DIR = Path("/tmp") / Path(ADK_TARGET_DIR)

print("SOURCE_DIR :", SOURCE_DIR)
print("TARGET_DIR :", TARGET_DIR)

#remove .env file
# def ignore_dotfiles(dir, files):
#     return [f for f in files if f.startswith('.')]

# Remove TARGET_DIR if it already exists
if TARGET_DIR.exists():
    shutil.rmtree(TARGET_DIR)
shutil.copytree(SOURCE_DIR, TARGET_DIR) #ignore=ignore_dotfiles

# Step 2: Restructure into proper Python package
PKG_DIR = TARGET_DIR / PKG_NAME
DIST_DIR = TARGET_DIR / "dist"

# Step 1: Move Python files into PKG_NAME / subfolder
if not PKG_DIR.exists():
    PKG_DIR.mkdir()
    for file in TARGET_DIR.glob("*.py"):
        if file.name != "setup.py":
            shutil.move(str(file), str(PKG_DIR / file.name))

# Step 2: Move folders (like 'tools') into the package dir
for sub in TARGET_DIR.iterdir():
    if sub.is_dir() and sub.name != PKG_NAME and sub.name != "dist":
        shutil.move(str(sub), str(PKG_DIR / sub.name))

# Step 3: Ensure __init__.py exists
(PKG_DIR / "__init__.py").touch(exist_ok=True)

# Step 4: Write or overwrite setup.py
setup_code = f"""
from setuptools import setup, find_packages

setup(
    name="{PKG_NAME}",
    version="0.1.0",
    description="",
    packages=find_packages(),
    include_package_data=True,
    install_requires=[
        "google-cloud-aiplatform[adk,agent_engines]",
    ],
    python_requires=">=3.8",
)
"""
with open(TARGET_DIR / "setup.py", "w") as f:
    f.write(setup_code)

# Step 5: Build and install the package
subprocess.run(["python", "-m", "build", "--outdir", str(DIST_DIR)], cwd=TARGET_DIR, check=True)
subprocess.run(["pip", "install", str(list(DIST_DIR.glob("*.whl"))[0])], check=True)


SOURCE_DIR : /tmp/ADK/adk_short_bot
TARGET_DIR : /tmp/packaged_agent/ADK/adk_short_bot
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for sdist...
running egg_info
creating adk_short_bot.egg-info
writing adk_short_bot.egg-info/PKG-INFO
writing dependency_links to adk_short_bot.egg-info/dependency_links.txt
writing requirements to adk_short_bot.egg-info/requires.txt
writing top-level names to adk_short_bot.egg-info/top_level.txt
writing manifest file 'adk_short_bot.egg-info/SOURCES.txt'
reading manifest file 'adk_short_bot.egg-info/SOURCES.txt'
writing manifest file 'adk_short_bot.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing adk_short_bot.egg-info/PKG-INFO
writing dependency_links to adk_short_bot.egg-info/dependency_links.txt
writing requirements to adk_short_bot.egg-info/requires.txt
writing top-level names to adk_short_bot.egg-info/top_level.txt
readi

  - setuptools >= 40.8.0
* Getting build dependencies for wheel...
running egg_info
writing adk_short_bot.egg-info/PKG-INFO
writing dependency_links to adk_short_bot.egg-info/dependency_links.txt
writing requirements to adk_short_bot.egg-info/requires.txt
writing top-level names to adk_short_bot.egg-info/top_level.txt
reading manifest file 'adk_short_bot.egg-info/SOURCES.txt'
writing manifest file 'adk_short_bot.egg-info/SOURCES.txt'
* Building wheel...
running bdist_wheel
running build
running build_py
creating build/lib/adk_short_bot
copying adk_short_bot/__init__.py -> build/lib/adk_short_bot
copying adk_short_bot/agent.py -> build/lib/adk_short_bot
copying adk_short_bot/prompt.py -> build/lib/adk_short_bot
creating build/lib/adk_short_bot/tools
copying adk_short_bot/tools/character_counter.py -> build/lib/adk_short_bot/tools
copying adk_short_bot/tools/__init__.py -> build/lib/adk_short_bot/tools
installing to build/bdist.linux-x86_64/wheel
running install
running install_lib
creat

CompletedProcess(args=['pip', 'install', '/tmp/packaged_agent/ADK/adk_short_bot/dist/adk_short_bot-0.1.0-py3-none-any.whl'], returncode=0)

In [12]:
# Upload to MinIO (optional)
from Framework import base_tools

for fpath in DIST_DIR.glob("*"):
    key = f"{PKG_NAME}/{fpath.name}"    
    base_tools.upload_filepath_to_datalake(str(fpath), 'datalake', f'Agent/{key}')
    
    print(f"☁️ Uploaded {fpath.name} to s3://datalake/Agent/{key}")

UPLOAD FILE TO S3://datalake/Agent/adk_short_bot/adk_short_bot-0.1.0-py3-none-any.whl SUCCESSFULY!
☁️ Uploaded adk_short_bot-0.1.0-py3-none-any.whl to s3://datalake/Agent/adk_short_bot/adk_short_bot-0.1.0-py3-none-any.whl
UPLOAD FILE TO S3://datalake/Agent/adk_short_bot/adk_short_bot-0.1.0.tar.gz SUCCESSFULY!
☁️ Uploaded adk_short_bot-0.1.0.tar.gz to s3://datalake/Agent/adk_short_bot/adk_short_bot-0.1.0.tar.gz


In [13]:
import os
import boto3
from pathlib import Path
import subprocess
from botocore.config import Config


# MINIO_KEY = f"Agent/{PKG_NAME}/{PKG_NAME}-0.1.0-py3-none-any.whl" 
MINIO_KEY = f"Agent/{PKG_NAME}/{PKG_NAME}-0.1.0.tar.gz"
LOCAL_PKG_PATH = Path("/tmp/registered_agent") / Path(MINIO_KEY).name

print("MINIO_KEY :", MINIO_KEY)
print("LOCAL_PKG_PATH :", LOCAL_PKG_PATH)

# === Ensure download directory exists ===
LOCAL_PKG_PATH.parent.mkdir(parents=True, exist_ok=True)

# === Download from MinIO ===
s3 = base_tools.register_datalake()

print(f"⬇️  Downloading {MINIO_KEY} from MinIO...")
s3.download_file("datalake", MINIO_KEY, str(LOCAL_PKG_PATH))
print(f"✅ Downloaded to: {LOCAL_PKG_PATH}")

# === Uninstall package if found ===
print(f"⛔ Uninstalling any existing versions of {PKG_NAME}...")
subprocess.run(["pip", "uninstall", PKG_NAME, "-y"], check=False)

# === Install package === persistence as long as container don't force to shutdown
print(f"📦 Installing {LOCAL_PKG_PATH.name}...")
subprocess.run(["pip", "install", str(LOCAL_PKG_PATH)], check=True)
print("✅ Package installed.")


MINIO_KEY : Agent/adk_short_bot/adk_short_bot-0.1.0.tar.gz
LOCAL_PKG_PATH : /tmp/registered_agent/adk_short_bot-0.1.0.tar.gz
⬇️  Downloading Agent/adk_short_bot/adk_short_bot-0.1.0.tar.gz from MinIO...
✅ Downloaded to: /tmp/registered_agent/adk_short_bot-0.1.0.tar.gz
⛔ Uninstalling any existing versions of adk_short_bot...
Found existing installation: adk-short-bot 0.1.0
Uninstalling adk-short-bot-0.1.0:
  Successfully uninstalled adk-short-bot-0.1.0
📦 Installing adk_short_bot-0.1.0.tar.gz...
Processing /tmp/registered_agent/adk_short_bot-0.1.0.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  DEPRECATION: Building 'adk-short-bot' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'adk-short-bot'. Discussion can be found at https://github.com/pypa/pip/issues/6334


  Created wheel for adk-short-bot: filename=adk_short_bot-0.1.0-py3-none-any.whl size=2908 sha256=2c6a04bbb9f73817339991be2c4653f976453add95360a85eb7396998d512f2e
  Stored in directory: /home/jovyan/.cache/pip/wheels/e8/81/38/6a42e9cf18167bb88506db68f99d22951fc3f55d54e2ab2eca
Successfully built adk-short-bot
✅ Package installed.


## Test Run

In [ ]:
# === Use your agent ===
import uuid
import os
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
import importlib

module = importlib.import_module(PKG_NAME+".agent")
root_agent = getattr(module, "root_agent")

load_dotenv("/home/jovyan/notebooks/.env")

# Create a new session service to store state
session_service_stateful = InMemorySessionService()

# Create a NEW session
APP_NAME = "TEST-RUN-APP"
USER_ID = "TEST-USER"
SESSION_ID = str(uuid.uuid4())
stateful_session = session_service_stateful.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)
print("CREATED NEW SESSION:")
print(f"\tSession ID: {SESSION_ID}")

runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service_stateful,
)

new_message = types.Content(
    role="user", parts=[types.Part(text="hey what you can do for me?")]
)

for event in runner.run(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=new_message,
):
    if event.is_final_response():
        if event.content and event.content.parts:
            print(f"Final Response: {event.content.parts[0].text}")


